## Solution to Open AI's [Cart Pole V0] (https://gym.openai.com/envs/CartPole-v0)

### Based of [Policy-based Agents Tutorial](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724) and [Andrej Karpathy Gist](https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5)

In [2]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

### Setup gym env

In [3]:
import gym
env = gym.make('CartPole-v0')

[2017-05-29 18:52:29,002] Making new env: CartPole-v0


### Test dimensionality of the observation space

In [4]:
env.reset()
sample_observation, _, _, info = env.step(1)
print sample_observation

[ 0.03550155  0.16023649 -0.03228949 -0.32001057]


### Setup Neural Network


#### Hyper Params

In [9]:
tf.reset_default_graph()


input_size = len(sample_observation)
learning_rate = 0.01

# I am going to be using two layer NL
hidden_layer_size = 20
w1_size = [input_size, hidden_layer_size]
w2_size = [hidden_layer_size, 1]

# Define tensors
observation = tf.placeholder(tf.float32, [None, input_size], name="input_obs")
W1 = tf.get_variable("W1", shape=w1_size, initializer=tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(observation, W1))

W2 = tf.get_variable("W2", shape=w2_size, initializer=tf.contrib.layers.xavier_initializer())
probability = tf.nn.sigmoid(tf.matmul(layer1, W2))

training_vars = tf.trainable_variables()
fake_labels = tf.placeholder(tf.float32,[None,1], name="fake_labels")
advantages = tf.placeholder(tf.float32,name="advantages")

loglik = tf.log(fake_labels*(fake_labels - probability) + (1 - fake_labels)*(fake_labels + probability))
loss = -tf.reduce_mean(loglik * advantages)
updated_grads = tf.gradients(loss, training_vars)


adam = tf.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
W1Grad = tf.placeholder(tf.float32,name="batch_grad1") # Placeholders to send the final gradients through when we update.
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,training_vars))